In [1]:
import numpy as np
import pandas as pd

In [2]:
def sim_Jaccard (str1,str2) :
    set1 = set( str1.lower().replace(';',' ').replace(',',' ').replace('.',' ').replace(':',' ').replace('&',' ').
               replace('/',' ').replace('\'',' ').replace('(author)',' ').replace('(joint author)',' ').split() )
    set2 = set( str2.lower().replace(';',' ').replace(',',' ').replace('.',' ').replace(':',' ').replace('&',' ').
               replace('/',' ').replace('\'',' ').replace('(author)',' ').replace('(joint author)',' ').split() )
    return len(set1&set2)/len(set1|set2)

In [3]:
def MV(df,indexK='isbn',answer='author',withWeight=False,weight='fact_confidence'):
    df_mv = pd.DataFrame(columns=[indexK,answer])
    for indexV in df[indexK].unique():
        data_slice = df[df[indexK]==indexV]
        vote_dict = {}
        for index,row in data_slice.iterrows():
            flag = False
            for key in vote_dict.keys():
                if ( sim_Jaccard(key,row[answer])>=0.8 ):
                    flag = True
                    if(not withWeight):
                        vote_dict[key] += 1
                    else:
                        vote_dict[key] += float(row[weight])
                    break
            if (not flag):
                if(not withWeight):
                    vote_dict[row[answer]] = 1
                else:
                    vote_dict[row[answer]] = float(row[weight])
        vote_list = sorted(vote_dict.items(), key=lambda d:d[1],reverse=True)
        #print({indexK:indexV,answer:vote_list[0][0]})
        df_mv = df_mv.append({indexK:indexV,answer:vote_list[0][0]},ignore_index=True)
    return df_mv

In [4]:
dataframe = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
df_mv = MV(dataframe)

In [6]:
df_mv.set_index('isbn',inplace=True)
label = pd.read_csv('./DataSet/book/book_golden.txt',sep='\t',low_memory=False,names=['isbn','author'],header=None,index_col=0)

In [13]:
JudgeAccu(df_mv,label)

conklin, wm. arthur;  white, gregory b.;  cothren, chuck ;  williams, dwayne ;  davis, roger l.; vs Conklin, Art; Davis, Roger; Williams, Dwayne; Cothren, Chuck
suhanovs, dennis ; vs Dennis Suhanovs, Press Certification, Certification Press
white, gregory ;   vs White, Gregory B.
meadors, todd ;  schmidt, cheryl a.;   vs Meadors, Todd; Schmidt, Cheryl Ann
halpin, terry ;  evans, ken ;  hallock, pat ;  maclean, bill ;   vs Halpin, Terry
hoos, holger h.;   stutzle, thomas;   vs Hoos, Holger
strassner, john c.;   vs John Strassner
lacroix, zoe ;  critchlow, terence; vs Not Available
loshin, pete ;   vs Loshin, Peter
kennedy, james ;  eberhart, russell c.;   vs Eberhart, Russell
jones, karen sparck;  willett, peter ;   vs Edited by Jones, Karen Sparck Edited by Willett, Peter
widom, jennifer ;  ceri, stefano; vs Widom, Jennifer
erbschloe, michael ;   vs Erbschloe
etzel, michael ;  dickinson, karen ;   vs Etzel, Michael
steele, guy l.;   vs Guy Steele
shaw, paul d.;   vs Shaw, Paul
boyce, j

0.8779226190476191

In [ ]:
dataframe = pd.read_csv("./DataSet/vldbBook/data_golden/data_golden.txt",sep='\t')
mv_result = MV(dataframe)
mv_result.to_csv( './DataSet/vldbBook/MajorityVotingResult.txt' , sep='\t' , index=False )

In [12]:
def JudgeAccu(label,pred,pred_col='author'):
    score = 0
    for index,row in pred.iterrows():
        if not(index in label.index):
            print(index,'no answer')
            score += 0 
        elif sim_Jaccard(row[pred_col],label.loc[index][pred_col])>=0.75:
            score +=1
        else:
            print(row[pred_col],"vs",label.loc[index][pred_col])
            score += sim_Jaccard(row[pred_col],label.loc[index][pred_col])
    return score/len(pred)

In [ ]:
label = pd.read_csv('./DataSet/vldbBook/book_truth.txt',sep='\t',low_memory=False,index_col=0)
label.rename(columns={'isbn_10':'isbn','authors_truth':'author'},inplace=True)
pred = pd.read_csv('./DataSet/vldbBook/MajorityVotingResult.txt',sep='\t',low_memory=False,index_col=0)
print(JudgeAccu(label,pred))